In [1]:
import pandas as pd
import scipy
import numpy as np
import random
import copy
import math
from collections import defaultdict


#import timeit
#start = timeit.default_timer()

randomlist = []
demand = {}
random.seed(26) # Use the same random seed
for i in range(0,30):   # 20 depots
    n = random.randint(100,250) # Depot Capacity
    randomlist.append(n)

for i in range(30):
    demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(26) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,60, size=(sum,30))).astype(float)

random.seed(26) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(26) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(26) # Use the same random seed
doc = {}
for i in range(0,30):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,31)]
dk = df_tdkom.transpose()
costs = dk.to_dict()
#costs_c = df_tdkom.to_dict()
cols = sorted(demand.keys())

In [2]:
costs1=copy.deepcopy(costs)
costs2=copy.deepcopy(costs)
costs3=copy.deepcopy(costs)

for i in supply:
    mi=min(costs[i].values())
    #print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in demand :
    mi=10000
    for j in supply:
        if costs[j][i]<mi :
            mi=costs[j][i]
    for j in supply:
        costs3[j][i]=costs3[j][i]-mi
# print(costs3)

for i in demand:
    for j in supply:
        costs[j][i]= costs2[j][i]+costs3[j][i]

res = dict((k, defaultdict(int)) for k in costs)
ls=len(supply)


In [3]:
while supply and demand:
  sd={}
  # print(demand,supply)
  for x in demand:
      # print(x)
      l=[]
      # print(len(supply))
      for y in supply:
          l.append(costs[y][x])
      # print(l)
      if len(supply)>1:
          from scipy import stats
          sd[x]=scipy.stats.gmean(l)
      else:
          sd[x]=(list(supply.values()))[0]

  for y in supply :
      l=[]
      for x in demand:
          l.append(costs[y][x])
      if len(demand)>1:
          from scipy import stats
          sd[y]=scipy.stats.gmean(l)
      else:
          sd[y]=(list(demand.values()))[0]
      # sd[y]=stdev(l)
  # print(sd)
  ma=max(sd.values())
  l=[]
  for ke in sd:
      if sd[ke]==ma:
          l.append(ke)
  ind=l[0]


  if demand.get(ind)!=None:
      mi=10000
      mind="S"
      for y in supply:
          if costs[y][ind]<mi:
              mi=costs[y][ind]
              mind=y
      v=min(supply[mind],demand[ind])
      supply[mind]-=v
      demand[ind]-=v
      res[mind][ind]+=v
      if supply[mind]==0:
          del supply[mind]
      if demand[ind]==0:
          del demand[ind]
  else:
      mi=10000
      mind="S"
      # print(ind)
      for y in demand:
          # print(ind,y)
          if costs[ind][y]<mi:
              mi=costs[ind][y]
              mind=y
      v=min(supply[ind],demand[mind])
      supply[ind]-=v
      demand[mind]-=v
      res[ind][mind]+=v
      if supply[ind]==0:
          del supply[ind]
      if demand[mind]==0:
          del demand[mind]

  # break

In [4]:
cost = 0
for g in sorted(costs1):
    # print (g, "\t",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("\t",)
    # print()
# print ("\n\nTotal Cost = ", cost)

In [5]:
cost

791719.6550284085